In [1]:
from openhunt.logparser import winlogbeat
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
win = winlogbeat()
spark = SparkSession.builder.appName("Process").config("spark.sql.caseSensitive", "True").getOrCreate()

In [3]:
process_injection = win.extract_nested_fields("empire_psinject_2019-05-18200432.json",spark)

[+] Processing a Spark DataFrame..
[+] Reading Mordor file..
[+] Processing Data from Winlogbeat version 6..
[+] DataFrame Returned !


In [4]:
process_injection.createOrReplaceTempView("process_injection")

In [5]:
ReflectiveDLL_ProcessInjection = spark.sql(

'''
SELECT 
    b.computer_name,
    ltrim('0x', b.GrantedAccess) as granted_access,
    a.SourceImage,
    a.SourceProcessGuid,
    a.NewThreadId, 
    a.TargetProcessId,
    a.TargetImage
FROM process_injection a
JOIN process_injection b 
    ON a.SourceProcessGuid = b.SourceProcessGUID  
    AND b.channel = "Microsoft-Windows-Sysmon/Operational"
    AND b.event_id = 10
    AND a.TargetProcessGuid = b.TargetProcessGUID
WHERE a.channel = "Microsoft-Windows-Sysmon/Operational"
    AND a.event_id = 8
'''
)

In [6]:
df = ReflectiveDLL_ProcessInjection.select("computer_name", conv(ReflectiveDLL_ProcessInjection.granted_access, 16, 10).alias("granted_access"), "SourceImage", "SourceProcessGuid", "NewThreadId", "TargetProcessId", "TargetImage")

In [7]:
df.createOrReplaceTempView("df")

In [8]:
eid_8_10_minimal_privileges = spark.sql(
'''
SELECT * FROM df WHERE ((INT(granted_access) & 5178) == 5178) -- 5178 is decimal for 0x143A. The minimal privileges you need to access process handle
'''
).show(1,False)

+---------------+--------------+---------------------------------------------------------+--------------------------------------+-----------+---------------+-------------------------------+
|computer_name  |granted_access|SourceImage                                              |SourceProcessGuid                     |NewThreadId|TargetProcessId|TargetImage                    |
+---------------+--------------+---------------------------------------------------------+--------------------------------------+-----------+---------------+-------------------------------+
|HR001.shire.com|2047999       |C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe|{03ba39f5-50b2-5ce0-0000-00109995c501}|9180       |7924           |C:\Windows\System32\notepad.exe|
+---------------+--------------+---------------------------------------------------------+--------------------------------------+-----------+---------------+-------------------------------+
only showing top 1 row

